In [1]:
import numpy as np
import cv2
import sys
from PIL import ImageTk, Image, ImageDraw
import PIL
from tkinter import *
import glob

In [2]:
def draw(case, name):
    width = 200  # canvas width
    height = 200 # canvas height
    center = height//2
    white = (255, 255, 255) # canvas back
    
    def save():
        if case == 'draw_for_train_x':
            PATH = 'train\\x\\'
            output_image.save(PATH+str(name)+'.jpg')
        if case == 'draw_for_train_o':
            PATH = 'train\\o\\'
            output_image.save(PATH+str(name)+'.jpg')
        if case == 'draw_for_test':
            PATH = 'test\\'
            output_image.save(PATH+'test'+'.jpg')
    
    def paint(event):
        x1, y1 = (event.x - 1), (event.y - 1)
        x2, y2 = (event.x + 1), (event.y + 1)
        canvas.create_oval(x1, y1, x2, y2, fill="black",width=5)
        draw.line([x1, y1, x2, y2],fill="black",width=5)

    master = Tk()

    # create a tkinter canvas to draw on
    canvas = Canvas(master, width=width, height=height, bg='white')
    canvas.pack()

    # create an empty PIL image and draw object to draw on
    output_image = PIL.Image.new("RGB", (width, height), white)
    draw = ImageDraw.Draw(output_image)
    canvas.pack(expand=YES, fill=BOTH)
    canvas.bind("<B1-Motion>", paint)

    # add a button to save the image
    button=Button(text="save",command=save)
    button.pack()

    master.mainloop()

In [3]:
def train():

    print("IN MAIN")
    
    w = np.zeros(40000)
    b = 0
    
    folders_x = glob.glob('train\\x')
    
    paths_x = []

    for folder in folders_x:
        for f in glob.glob(folder+'/*.jpg'):
            paths_x.append(f)

    read_images_x = []

    for path_x in paths_x:
        a = cv2.imread(path_x, cv2.IMREAD_GRAYSCALE)
        a = cv2.resize(a, (200, 200))
        read_images_x.append(a)

    arr_img_x = np.zeros((len(read_images_x), 40000))
    y_x = np.zeros(len(arr_img_x))

    for i in range(len(read_images_x)):
        arr_img_x[i] = read_images_x[i].ravel()

    for i in range(len(arr_img_x)):
        for j in range(len(arr_img_x[i])):
            if(arr_img_x[i][j] >= 128):
                arr_img_x[i][j] = -1
            else:
                arr_img_x[i][j] = 1
                
        y_x[i] = 1
                
    folders_o = glob.glob('train\\o')
    
    paths_o = []

    for folder in folders_o:
        for f in glob.glob(folder+'/*.jpg'):
            paths_o.append(f)

    read_images_o = []

    for path_o in paths_o:
        a = cv2.imread(path_o, cv2.IMREAD_GRAYSCALE)
        a = cv2.resize(a, (200, 200))
        read_images_o.append(a)

    arr_img_o = np.zeros((len(read_images_o), 40000))
    y_o = np.zeros(len(arr_img_o))

    for i in range(len(read_images_o)):
        arr_img_o[i] = read_images_o[i].ravel()

    for i in range(len(arr_img_o)):
        for j in range(len(arr_img_o[i])):
            if(arr_img_o[i][j] >= 128):
                arr_img_o[i][j] = -1
            else:
                arr_img_o[i][j] = 1
                
        y_o[i] = -1
        
    arr_img = np.concatenate((arr_img_x, arr_img_o))
    y = np.concatenate((y_x, y_o))
    
    for i in range(len(w)):
        for j in range(len(arr_img)):
            w[i] = w[i] + arr_img[j][i] * y[j]
            
    for i in range(len(arr_img)):
        b = b + y[i]
            
    
    return w, b

In [4]:
def test(w, b):
    
    img = cv2.imread('test/test.jpg', cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (200, 200))
    
    print('Putting Image through the net')
    
    sigma = 0
    yIN = 0

    img = img.ravel()
    
    for i in range(len(img)):
        if(img[i] >= 128):
            img[i] = -1
        else:
            img[i] = 1
                        
    yIN = b + np.matmul(img, w)
            
    if (yIN <= -15000000.0):
        return 1
    else:
        return -1

In [5]:
t_or_t = input('Draw for : 1-Train OR 2-Test ? (Write the number)\n')

if(int(t_or_t) == 1):
    x_or_o = input('\nDraw : 1-X OR 2-O ? (Write the number)\n')
    
    if(int(x_or_o) == 1):
        aar_of_drawing_for_train_x = []
        num_of_drawing_for_train_x = input('\nHow many images do you want to draw?\n')
        
        for i in range(0, int(num_of_drawing_for_train_x)):
            n = input('\nWrite a name for file. \n EXCEPT:[x1,x2,x3,x4,x5,x6,x7,x8,x9,x10]\n')
            aar_of_drawing_for_train_x.append(n)
            
        for i in range(0, int(num_of_drawing_for_train_x)):
            draw('draw_for_train_x', aar_of_drawing_for_train_x[i])
        
    elif(int(x_or_o) == 2):
        aar_of_drawing_for_train_o = []
        num_of_drawing_for_train_o = input('\nHow many images do you want to draw?\n')
        
        for i in range(0, int(num_of_drawing_for_train_o)):
            n = input('\nWrite a name for file.\nEXCEPT:[o1,o2,o3,o4,o5,o6,o7,o8,o9,o10]\n')
            aar_of_drawing_for_train_o.append(n)
            
        for i in range(0, int(num_of_drawing_for_train_o)):
            draw('draw_for_train_o', aar_of_drawing_for_train_o[i])
        
elif(int(t_or_t) == 2):
    
    draw('draw_for_test', 'test')

    result_train = train()
    Result = test(result_train[0], result_train[1])
    
    if(Result == 1):
        print("\nThe image given is X")
    elif(Result == -1):
        print("\nThe image given is O")
    else:
        print("\nERROR")
    
else:
    print('ERROR')

Draw for : 1-Train OR 2-Test ? (Write the number)
2
IN MAIN
Putting Image through the net

The image given is X
